Clean the raw weather data

In [83]:
import pandas as pd
import os
import sys
sys.path.append('../scripts') #Add scripts folder to path

from utilities import (
    profile_dataframe_overview,
    summary_stats,
    check_constant_columns,
    check_empty_columns
)

cleaning the raw weather data previously saved as a CSV for data processing. 

- Converting the `time` column into a proper `timestamp`
- Dropping the original `time` column
- Removing duplicate or fully empty rows
- Reordering columns for consistency

The cleaned weather data is saved in `data/processed/weather_data_cleaned.csv` and will be ready for merging with thermostat data.

In [84]:
# Define input/output paths
raw_path = "../data/raw/weather_data_hourly.csv"
processed_dir = "../data/processed"
os.makedirs(processed_dir, exist_ok=True)
cleaned_path = os.path.join(processed_dir, "weather_data_cleaned.csv")



In [85]:
# Load raw weather data . This dataset contains external weather parameters such as temperature, humidity, and wind speed at hourly intervals.

weather_df = pd.read_csv(raw_path)
# print("Current Working Directory:", os.getcwd())
# print(weather_df)

### profile_dataframe_overview()  BASIC OVERVIEW
This function provides a structured overview of any DataFrame. It displays:

- The number of rows and columns
- Column names and data types
- DataFrame info summary (including memory usage)
- Missing values per column (as percentages)
- Unique values per column
- Sample preview of the first and last N rows

In [86]:
profile_dataframe_overview(weather_df)


 Basic Structure
----------------------------------------
Rows: 10224, Columns: 4
Column Names: ['time', 'temperature_2m', 'relative_humidity_2m', 'windspeed_10m']

 Data Types
----------------------------------------
time                     object
temperature_2m          float64
relative_humidity_2m      int64
windspeed_10m           float64
dtype: object

 DataFrame Summary Info
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10224 entries, 0 to 10223
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   time                  10224 non-null  object 
 1   temperature_2m        10224 non-null  float64
 2   relative_humidity_2m  10224 non-null  int64  
 3   windspeed_10m         10224 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 319.6+ KB

 Missing Data Overview (%)
----------------------------------------
No missing values.

 Uniqu

### Summary_stats   DISPLAY STATS SUMMARY

This function returns standard summary statistics for all numeric columns in the DataFrame:

- count, mean, std
- min, 25th percentile, median (50%), 75th percentile, max


In [87]:
summary_statistics = summary_stats(weather_df)
print(summary_statistics)


 Summary Statistics:
                        count   mean    std   min   25%   50%   75%    max
temperature_2m        10224.0  13.99  10.65 -20.3   6.4  16.3  22.1   35.6
relative_humidity_2m  10224.0  70.88  17.32  19.0  58.0  72.0  85.0  100.0
windspeed_10m         10224.0  11.09   5.66   0.0   7.1  10.1  14.4   55.7


### check_constant_columns

This function identifies columns where all rows have the same value (e.g., a column that always says "My Ecobee" or 0 values).

Such columns are usually not informative and can be dropped to simplify the dataset.

In [88]:
const_columns = check_constant_columns(weather_df)
print("\n Constant Columns:", const_columns)


 Constant Columns: []

 Constant Columns: []


### check_empty_columns

This function returns a list of columns that are completely empty (i.e., 100% null values).

These columns typically be removed unless or populate them later.

In [89]:
empty_columns = check_empty_columns(weather_df)
print("\n Empty Columns:", empty_columns)


 Empty Columns: []

 Empty Columns: []


In [90]:
# Drop constant and empty columns
# weather_df.drop(columns=const_columns + empty_columns, inplace=True, errors='ignore')
#print("Dataframe after constant and empty columns drop: \n", weather_df)

In [91]:
# Drop duplicates
# weather_df.drop_duplicates(inplace=True)

print("Duplicate rows:", weather_df.duplicated().sum())

Duplicate rows: 0


### Rename Columns for Consistency

To maintain consistency and ease of merging with the thermostat dataset later, renamed the columns using more descriptive and uniform names:

- `time` → `timestamp`
- `temperature_2m` → `outdoor_temp_c` 
- `relative_humidity_2m` → `outdoor_humidity`
- `windspeed_10m` → `wind_speed_kmh`


In [92]:
# Rename columns for consistency and clarity
weather_df.rename(columns={
    'time': 'timestamp',
    'temperature_2m': 'outdoor_temp_c',  # temp in °C temporarily
    'relative_humidity_2m': 'outdoor_humidity',
    'windspeed_10m': 'wind_speed_kmh'
}, inplace=True)

column_names = weather_df.columns.tolist()
print(column_names)

['timestamp', 'outdoor_temp_c', 'outdoor_humidity', 'wind_speed_kmh']


### Convert Time to Timestamp Format
The `time` column is converted to `datetime` format and renamed as `timestamp` for compatibility with the thermostat dataset.

In [93]:

# Convert 'time' to timestamp

weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
print(weather_df['timestamp'])

0       2024-05-01 00:00:00
1       2024-05-01 01:00:00
2       2024-05-01 02:00:00
3       2024-05-01 03:00:00
4       2024-05-01 04:00:00
                ...        
10219   2025-06-30 19:00:00
10220   2025-06-30 20:00:00
10221   2025-06-30 21:00:00
10222   2025-06-30 22:00:00
10223   2025-06-30 23:00:00
Name: timestamp, Length: 10224, dtype: datetime64[ns]


### Convert Temperature from Celsius to Fahrenheit
Since the thermostat dataset reports temperatures in Fahrenheit, convert the `outdoor_temp_c` column from Celsius to Fahrenheit and rename to `outdoor_temp_f`. Then drop the `outdoor_temp_c`.

In [94]:
#weather_df['outdoor_temp_f'] = (weather_df['outdoor_temp_c'] * 9/5) + 32
#column_names = weather_df.columns.tolist()
#print(column_names)

#weather_df.drop(columns=['outdoor_temp_c'], inplace=True)
#print(column_names)

if 'outdoor_temp_c' in weather_df.columns:
    weather_df['outdoor_temp_f'] = (weather_df['outdoor_temp_c'] * 9/5) + 32
    #weather_df['outdoor_temp_f'] = weather_df['outdoor_temp_f'].round(2)
    weather_df.drop(columns=['outdoor_temp_c'], inplace=True)
else:
    print("Column 'outdoor_temp_c' not found. Skipping temperature conversion.")

print(weather_df)

                timestamp  outdoor_humidity  wind_speed_kmh  outdoor_temp_f
0     2024-05-01 00:00:00                75             6.1           58.46
1     2024-05-01 01:00:00                80             5.5           56.30
2     2024-05-01 02:00:00                83             4.8           55.04
3     2024-05-01 03:00:00                86             7.4           54.14
4     2024-05-01 04:00:00                88             9.3           54.14
...                   ...               ...             ...             ...
10219 2025-06-30 19:00:00                78            11.9           80.42
10220 2025-06-30 20:00:00                82             7.2           79.52
10221 2025-06-30 21:00:00                91             4.5           77.18
10222 2025-06-30 22:00:00                90             5.9           76.64
10223 2025-06-30 23:00:00                92             5.9           75.74

[10224 rows x 4 columns]


### Reorder Columns
Rearrange the columns to place `timestamp` as the first column (if needed), followed by the weather variables. This improves readability and aligns with the thermostat dataset.

In [95]:

# Reorder columns - timestamp first
cols = ['timestamp'] + [col for col in weather_df.columns if col != 'timestamp']
weather_df = weather_df[cols]

### Outlier Detection

Identifying potential outliers using the Interquartile Range (IQR) method. This helps in detecting anomalous values that might distort analysis or model performance.

.quantile() takes a float value between 0 and 1 to specify the desired quantile.
Q1 = .25 * (N-1) where N is number of datapoints in the list
Q3 = .75 * (N-1)
interpolation:
Q = Xi + f * (Xi+1 - Xi) 
Xi - Value at lower index
Xi+1 - Value at the upper index
f - fractional distance between lower and upper index (e.g 0.25, .75)

In [96]:
Q1 = weather_df.quantile(0.25) # Returns the value below which 25% of the data lies (Q1)
Q3 = weather_df.quantile(0.75) # Returns the value below which 75% of the data lies (Q3)
IQR = Q3 - Q1
outliers = ((weather_df < (Q1 - 1.5 * IQR)) | (weather_df > (Q3 + 1.5 * IQR))) # outliers threshold
print(outliers.sum())  # No. of outliers per column

# Filtering out the outliers
outlier_rows = weather_df[outliers.any(axis=1)]
print(outlier_rows[['timestamp', 'wind_speed_kmh', 'outdoor_temp_f']])

timestamp             0
outdoor_humidity      0
wind_speed_kmh      195
outdoor_temp_f       22
dtype: int64
               timestamp  wind_speed_kmh  outdoor_temp_f
254  2024-05-11 14:00:00            26.9           67.46
255  2024-05-11 15:00:00            25.5           67.46
256  2024-05-11 16:00:00            28.3           66.56
257  2024-05-11 17:00:00            26.9           66.74
258  2024-05-11 18:00:00            26.2           65.66
...                  ...             ...             ...
9160 2025-05-17 16:00:00            26.8           68.54
9161 2025-05-17 17:00:00            27.9           69.08
9253 2025-05-21 13:00:00            25.8           64.58
9257 2025-05-21 17:00:00            26.6           65.30
9258 2025-05-21 18:00:00            26.7           64.40

[217 rows x 3 columns]


In [97]:
# Tagging the outliers to handle or visualize later 

weather_df['temp_outlier'] = ((weather_df['outdoor_temp_f'] < (Q1['outdoor_temp_f'] - 1.5 * IQR['outdoor_temp_f'])) |
                               (weather_df['outdoor_temp_f'] > (Q3['outdoor_temp_f'] + 1.5 * IQR['outdoor_temp_f'])))

print(weather_df['temp_outlier'])

0        False
1        False
2        False
3        False
4        False
         ...  
10219    False
10220    False
10221    False
10222    False
10223    False
Name: temp_outlier, Length: 10224, dtype: bool


In [98]:

# Save cleaned version
weather_df.to_csv(cleaned_path, index=False, float_format="%.2f")
print(f" Cleaned weather data saved to: {cleaned_path}")

 Cleaned weather data saved to: ../data/processed/weather_data_cleaned.csv


In [99]:
#print(weather_df.dtypes)